In [ ]:
# Importar pandas para la manipulación de datos
import pandas as pd

# URL del conjunto de datos
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Definir los nombres de las columnas, ya que el archivo CSV original no los incluye
columnas = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
            "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

# Cargar el dataset utilizando pandas
df = pd.read_csv(url, names=columnas)

# Mostrar las primeras 5 filas del dataset para una inspección inicial
print("Primeras filas del DataFrame:")
df.head()

### 1. Análisis Exploratorio de Datos (EDA) - Parte 1: Entendimiento Básico

Una vez cargados los datos, el siguiente paso crucial es el Análisis Exploratorio de Datos (EDA). Esto nos permite entender la estructura, calidad y las principales características del conjunto de datos.

Realizaremos las siguientes comprobaciones:
1.  **Forma del dataset:** Para conocer el número de filas (observaciones) y columnas (características).
2.  **Distribución de la variable objetivo (`Outcome`):** Es vital saber si las clases (diabetes sí/no) están balanceadas o no. Un desbalanceo puede afectar el rendimiento del modelo y requerir técnicas especiales.
3.  **Estadísticas descriptivas:** Nos proporcionan un resumen numérico de cada característica (media, desviación estándar, mínimos, máximos, cuartiles), lo cual ayuda a identificar rangos de valores y posibles anomalías.

**¿Por qué este paso es importante para un empresario?**
El EDA revela la calidad de los datos y si son adecuados para el problema que queremos resolver. Por ejemplo, si la variable objetivo está muy desbalanceada (pocos casos de diabetes), el modelo podría tener dificultades para aprender a identificarlos. Conocer esto de antemano permite planificar estrategias para mitigar estos problemas.

In [ ]:
# Obtener las dimensiones del dataset (filas, columnas)
print("Forma del dataset (filas, columnas):", df.shape)
print("\n--------------------------------------------------\n")

# Analizar la distribución de la variable objetivo 'Outcome'
# Outcome: 0 significa sin diabetes, 1 significa con diabetes
print("Distribución de la variable objetivo 'Outcome':")
print(df["Outcome"].value_counts())
print("\nPorcentaje de cada clase:")
print(df["Outcome"].value_counts(normalize=True) * 100)
print("\n--------------------------------------------------\n")

# Obtener estadísticas descriptivas para cada columna numérica
print("Estadísticas descriptivas del dataset:")
df.describe().transpose() # Transpose para mejor visualización

**Interpretación de los resultados iniciales:**
*   El dataset tiene 768 observaciones y 9 características (incluyendo la variable objetivo).
*   La variable 'Outcome' muestra un desbalanceo: aproximadamente 65% de los casos son '0' (sin diabetes) y 35% son '1' (con diabetes). Esto es importante tenerlo en cuenta para la evaluación del modelo y posibles técnicas de remuestreo.
*   Al observar las estadísticas descriptivas, notamos que algunas columnas como `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` y `BMI` tienen valores mínimos de 0. Fisiológicamente, estos valores no pueden ser cero para una persona viva. Esto sugiere que los ceros en estas columnas podrían estar representando datos faltantes o erróneos.

### 2. Preprocesamiento de Datos - Parte 1: Manejo de Valores Anómalos (Ceros)

Como identificamos en el EDA, ciertas columnas tienen valores '0' que no son realistas. Procederemos a reemplazar estos ceros con `NaN` (Not a Number), que es la forma estándar de representar valores faltantes en pandas. Esto nos permitirá luego aplicar estrategias de imputación adecuadas.

Las columnas a tratar son: `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, `BMI`.

**¿Por qué este paso es importante para un empresario?**
La calidad de los datos de entrada impacta directamente la calidad del modelo predictivo. Ignorar valores anómalos o incorrectos puede llevar a un modelo que aprenda patrones erróneos y, por lo tanto, tome decisiones incorrectas. Corregir estos datos es un paso esencial hacia la fiabilidad.

In [ ]:
# Importar numpy para utilizar np.nan (Not a Number)
import numpy as np

# Lista de columnas donde el valor 0 es fisiológicamente implausible y probablemente indica un dato faltante
columnas_con_ceros_implausibles = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# Reemplazar 0 con NaN en las columnas especificadas
df[columnas_con_ceros_implausibles] = df[columnas_con_ceros_implausibles].replace(0, np.nan)

# Verificar la cantidad de valores nulos (NaN) por columna después del reemplazo
print("Cantidad de valores nulos por columna después de reemplazar ceros:")
df.isnull().sum()

**Observación:**
Ahora vemos que las columnas `Insulin` y `SkinThickness` tienen una cantidad considerable de valores faltantes (casi la mitad y un tercio de los datos, respectivamente). `Glucose`, `BloodPressure` y `BMI` también tienen algunos valores faltantes.

### 2. Preprocesamiento de Datos - Parte 2: Imputación de Valores Faltantes

Los valores faltantes deben ser manejados antes de entrenar un modelo de Machine Learning, ya que la mayoría de los algoritmos no los aceptan. Una estrategia común es la **imputación**, que consiste en reemplazar los `NaN` con un valor estadístico calculado a partir de los datos existentes en esa columna (e.g., la media o la mediana).

Aquí, optaremos por imputar los valores faltantes con la **media** de cada columna respectiva. Es importante calcular esta media *solo* con los datos de entrenamiento en un escenario real para evitar fuga de datos (data leakage) del conjunto de prueba. Sin embargo, para simplificar esta etapa inicial y dado que aún no hemos dividido los datos, aplicaremos la media global. Más adelante, al construir pipelines más robustos, la imputación se realizará correctamente dentro de los folds de validación cruzada o después de la división entrenamiento/prueba.

**¿Por qué este paso es importante para un empresario?**
No podemos simplemente eliminar todas las filas con datos faltantes, especialmente si son muchas, ya que perderíamos información valiosa. La imputación nos permite conservar la mayor cantidad de datos posible, rellenando los vacíos de manera lógica para que el modelo pueda trabajar con un dataset completo.

In [ ]:
# Imputar los valores NaN con la media de cada columna
# Es importante notar que en un flujo de trabajo riguroso, esta media se calcularía SOLO en el conjunto de entrenamiento.
# Por ahora, para mantener la simplicidad en esta etapa exploratoria, usamos la media del dataset completo.
df[columnas_con_ceros_implausibles] = df[columnas_con_ceros_implausibles].fillna(df[columnas_con_ceros_implausibles].mean())

# Verificar que no queden valores nulos
print("Cantidad de valores nulos por columna después de la imputación:")
df.isnull().sum()

**Comentario:**
Todos los valores nulos han sido imputados. Ahora nuestro dataset está numéricamente completo y listo para la siguiente fase.

### 3. Visualización de Datos (EDA - Parte 2)

Ahora que los datos están limpios, podemos realizar algunas visualizaciones para entender mejor las distribuciones y relaciones entre las variables.

**¿Por qué este paso es importante para un empresario?**
Las visualizaciones traducen números complejos en gráficos comprensibles. Permiten identificar patrones, tendencias o anomalías de forma intuitiva, facilitando la comunicación de los hallazgos y la generación de hipótesis.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración general para los gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# 1. Distribución de la variable objetivo 'Outcome'
plt.figure(figsize=(6,4))
sns.countplot(x='Outcome', data=df)
plt.title('Distribución de la Variable Objetivo (Outcome)')
plt.xlabel('Outcome (0: No Diabetes, 1: Diabetes)')
plt.ylabel('Cantidad de Pacientes')
plt.show()

# 2. Histogramas de todas las características para ver sus distribuciones
df.hist(figsize=(15,10), bins=20)
plt.suptitle('Histogramas de Todas las Características', y=1.02, size=16)
plt.tight_layout() # Ajusta automáticamente los subplots para que encajen
plt.show()

# 3. Matriz de Correlación
# La correlación nos indica la relación lineal entre pares de variables.
# Valores cercanos a 1 o -1 indican una fuerte correlación positiva o negativa, respectivamente.
# Valores cercanos a 0 indican poca o ninguna correlación lineal.
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación de Características')
plt.show()

**Interpretación de las Visualizaciones:**
*   **Distribución de Outcome:** Confirma visualmente el desbalanceo de clases que vimos antes.
*   **Histogramas:** Nos muestran la forma de la distribución de cada variable. Por ejemplo, `Age` y `DiabetesPedigreeFunction` están sesgadas a la derecha. `Glucose` y `BMI` se asemejan más a una distribución normal.
*   **Matriz de Correlación:**
    *   `Glucose` tiene la correlación positiva más alta con `Outcome` (0.49), lo que sugiere que niveles más altos de glucosa están asociados con una mayor probabilidad de diabetes. Esto es médicamente esperado.
    *   `BMI` (0.31) y `Age` (0.24) también muestran correlaciones positivas moderadas con `Outcome`.
    *   `Pregnancies` y `Age` tienen una correlación positiva (0.54), lo cual es lógico.
    *   Es importante notar que la correlación mide relaciones *lineales*. Puede haber relaciones no lineales importantes que no se capturen aquí.

### 4. Preparación de Datos para el Modelo: División en Conjuntos de Entrenamiento y Prueba

Antes de entrenar cualquier modelo, es crucial dividir nuestro conjunto de datos en dos subconjuntos:
1.  **Conjunto de Entrenamiento (Training set):** Se utiliza para que el modelo aprenda los patrones de los datos.
2.  **Conjunto de Prueba (Test set):** Se utiliza para evaluar el rendimiento del modelo en datos que no ha visto antes. Esto nos da una medida más realista de cómo se comportará el modelo en el mundo real.

Separaremos las características (variables predictoras, `X`) de la variable objetivo (lo que queremos predecir, `y`).

Usaremos `train_test_split` de `sklearn.model_selection`.
*   `test_size=0.2`: Reservamos el 20% de los datos para el conjunto de prueba.
*   `random_state=42`: Asegura que la división sea la misma cada vez que ejecutamos el código, para reproducibilidad.
*   `stratify=y`: Es muy importante cuando hay desbalanceo de clases. Asegura que la proporción de la variable objetivo (`Outcome`) sea similar tanto en el conjunto de entrenamiento como en el de prueba.

**¿Por qué este paso es importante para un empresario?**
Entrenar y evaluar un modelo con los mismos datos llevaría a una sobreestimación de su rendimiento (el modelo se "aprende de memoria" los datos de entrenamiento). La división en entrenamiento y prueba simula un escenario real donde el modelo debe predecir sobre nueva información, dando una visión honesta de su verdadera capacidad predictiva y evitando falsas expectativas.

In [ ]:
# Importar la función para dividir los datos
from sklearn.model_selection import train_test_split

# Separar las características (X) de la variable objetivo (y)
X = df.drop("Outcome", axis=1) # X contiene todas las columnas excepto 'Outcome'
y = df["Outcome"]             # y contiene solo la columna 'Outcome'

# Dividir los datos en conjuntos de entrenamiento y prueba
# 80% para entrenamiento, 20% para prueba
# stratify=y asegura que la proporción de clases en 'Outcome' se mantenga en ambos conjuntos
# random_state para reproducibilidad
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar las dimensiones de los conjuntos resultantes
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)
print("\nProporción de clases en y_train:")
print(y_train.value_counts(normalize=True))
print("\nProporción de clases en y_test:")
print(y_test.value_counts(normalize=True))

**Comentario:**
Los datos se han dividido correctamente. La proporción de clases (diabetes/no diabetes) es similar en los conjuntos de entrenamiento y prueba gracias al parámetro `stratify=y`.

### 5. Entrenamiento de un Modelo Base: Random Forest

Ahora vamos a entrenar nuestro primer modelo. Utilizaremos `RandomForestClassifier`, un algoritmo popular y potente que suele dar buenos resultados "de fábrica" (sin mucha optimización de hiperparámetros). Es un modelo de ensamblaje que construye múltiples árboles de decisión y combina sus predicciones.

1.  **Inicializar el modelo:** Creamos una instancia del clasificador. `random_state=42` para reproducibilidad.
2.  **Entrenar el modelo:** Usamos el método `fit()` con los datos de entrenamiento (`X_train`, `y_train`).
3.  **Realizar predicciones:** Usamos el método `predict()` con los datos de prueba (`X_test`).
4.  **Evaluar el modelo:** Usaremos `classification_report` de `sklearn.metrics` para obtener métricas clave como precisión, recall, F1-score y support.

**¿Por qué este paso es importante para un empresario?**
Este es el núcleo del proceso de Machine Learning: enseñar a una máquina a tomar decisiones o hacer predicciones basadas en datos históricos. La evaluación nos dirá qué tan bueno es el modelo en esta tarea. Métricas como:
*   **Precisión (Precision):** De todas las predicciones de "diabetes", ¿cuántas fueron correctas? Importante si el coste de un falso positivo es alto.
*   **Sensibilidad (Recall/Exhaustividad):** De todos los pacientes que realmente tienen diabetes, ¿a cuántos identificó correctamente el modelo? Crucial si no queremos pasar por alto casos de diabetes (minimizar falsos negativos).
*   **F1-Score:** Una media armónica de precisión y recall. Útil cuando hay desbalanceo de clases o cuando ambas métricas son importantes.
Estas métricas ayudan a entender el valor y las limitaciones del modelo en un contexto de negocio.

In [ ]:
# Importar el clasificador Random Forest y métricas de evaluación
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Inicializar el modelo Random Forest
# random_state para asegurar que los resultados sean reproducibles
modelo_rf_base = RandomForestClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
modelo_rf_base.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred_base = modelo_rf_base.predict(X_test)

# Evaluar el modelo
print("Reporte de Clasificación del Modelo Base (Random Forest):")
print(classification_report(y_test, y_pred_base))

print("Accuracy del Modelo Base:", accuracy_score(y_test, y_pred_base))

**Interpretación del Reporte de Clasificación (Modelo Base):**
*   **Clase 0 (No Diabetes):** Tiene alta precisión (0.79) y recall (0.87). Esto significa que el modelo es bueno identificando pacientes sin diabetes, y cuando predice "no diabetes", suele acertar.
*   **Clase 1 (Diabetes):** Tiene una precisión de 0.68 (cuando predice "diabetes", el 68% de las veces es correcto) y un recall de 0.56 (identifica al 56% de los pacientes que realmente tienen diabetes). El F1-score es 0.61.
*   **Accuracy (Exactitud General):** 0.76. El modelo clasifica correctamente el 76% de los casos en general.

El rendimiento para la clase minoritaria (diabetes=1) es notablemente más bajo, especialmente el recall. Esto es común en datasets desbalanceados. El modelo tiende a favorecer la clase mayoritaria.

### 6. Estrategias para Mejorar el Modelo: Validación Cruzada y Manejo del Desbalanceo con SMOTE

Para obtener una estimación más robusta del rendimiento del modelo y para abordar el desbalanceo de clases, implementaremos:

1.  **Validación Cruzada (Cross-Validation):**
    En lugar de una única división entrenamiento/prueba, la validación cruzada divide el conjunto de entrenamiento en múltiples "folds" (subconjuntos). El modelo se entrena en K-1 folds y se evalúa en el fold restante. Esto se repite K veces. El resultado final es el promedio de las métricas de rendimiento de cada fold. Esto da una medida más fiable de cómo el modelo generalizará a datos no vistos.
    Usaremos `StratifiedKFold` para asegurar que la proporción de clases se mantenga en cada fold, lo cual es crucial para datasets desbalanceados.

2.  **SMOTE (Synthetic Minority Over-sampling Technique):**
    Es una técnica para manejar el desbalanceo de clases. En lugar de simplemente duplicar instancias de la clase minoritaria (lo que podría llevar a sobreajuste), SMOTE crea nuevas instancias sintéticas de la clase minoritaria que son plausiblemente similares a las existentes. Esto ayuda al modelo a aprender mejor las características de la clase minoritaria.
    **Importante:** SMOTE debe aplicarse *solo* al conjunto de entrenamiento (o dentro de cada fold de entrenamiento en la validación cruzada) para evitar que el modelo vea información sintética derivada de lo que sería el conjunto de prueba.

Utilizaremos un `Pipeline` de `imblearn` (una librería que extiende `scikit-learn` para el manejo de desbalanceo). El pipeline aplicará SMOTE y luego entrenará el RandomForestClassifier secuencialmente dentro de cada fold de la validación cruzada.

**¿Por qué este paso es importante para un empresario?**
La validación cruzada proporciona una evaluación más confiable del rendimiento real del modelo, reduciendo la posibilidad de que un buen resultado en una única división de prueba sea solo por suerte. SMOTE ayuda a que el modelo sea más justo y efectivo en la predicción de la clase menos frecuente (en este caso, pacientes con diabetes), lo cual es a menudo el objetivo principal en problemas médicos.

In [ ]:
# Importar librerías necesarias para el pipeline, SMOTE y validación cruzada
from imblearn.pipeline import Pipeline as ImbPipeline # Renombrar para evitar conflicto con Pipeline de sklearn si se usa
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Definir el pipeline: 
# 1. Aplicar SMOTE para balancear las clases (solo en los datos de entrenamiento de cada fold)
# 2. Entrenar un RandomForestClassifier
pipeline_smote_rf = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('modelo_rf', RandomForestClassifier(random_state=42)) # Usaremos 'modelo_rf' para diferenciar del modelo base
])

# Definir la estrategia de validación cruzada
# n_splits=5 significa 5 folds. shuffle=True mezcla los datos antes de dividir.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizar la validación cruzada usando todo el dataset (X, y) para evaluar la estrategia general.
# En un escenario de reporte final, haríamos esto sobre X_train, y_train y luego evaluaríamos en X_test.
# Aquí, evaluamos el pipeline en su conjunto para ver el impacto de SMOTE.
# Usamos 'f1' como métrica de scoring, ya que es buena para clases desbalanceadas.
# Podríamos usar 'f1_weighted' o 'f1_macro' también.
scores = cross_val_score(pipeline_smote_rf, X, y, cv=cv, scoring="f1", n_jobs=-1)

print(f"F1-scores por fold con SMOTE + CV: {scores}")
print(f"F1 promedio con SMOTE + CV: {scores.mean():.4f}")
print(f"Desviación estándar del F1-score: {scores.std():.4f}")

**Interpretación de los Resultados de Validación Cruzada con SMOTE:**
El F1-score promedio obtenido mediante validación cruzada con SMOTE nos da una idea más robusta del rendimiento esperado del modelo. Este valor (alrededor de 0.64-0.66, dependiendo de la ejecución por `shuffle=True` si no se fija el `random_state` de SMOTE y RF dentro del pipeline para cada fold específicamente) es una mejor guía que el F1 de la clase positiva del modelo base (0.61).

Para el resto de las visualizaciones y la optimización, volveremos a entrenar un modelo (puede ser el base o uno con parámetros específicos) sobre `X_train`, `y_train` para luego evaluar sobre `X_test`.

### 7. Interpretación del Modelo y Visualizaciones Adicionales

Una vez que tenemos un modelo entrenado (usaremos el `modelo_rf_base` entrenado en `X_train`, `y_train` para estas visualizaciones, ya que es más sencillo de inspeccionar directamente que un pipeline complejo), es útil entender qué características son más importantes para sus predicciones y cómo se distribuyen sus errores.

1.  **Importancia de Características:** Random Forest puede decirnos qué características contribuyeron más a la toma de decisiones.
2.  **Matriz de Confusión:** Visualiza el rendimiento del clasificador. Muestra los Verdaderos Positivos (TP), Verdaderos Negativos (TN), Falsos Positivos (FP) y Falsos Negativos (FN).
3.  **Curva ROC y AUC:** La curva ROC (Receiver Operating Characteristic) grafica la tasa de verdaderos positivos contra la tasa de falsos positivos para diferentes umbrales de clasificación. El AUC (Area Under the Curve) es una medida única del rendimiento general del clasificador (un valor más cercano a 1 es mejor).
4.  **Distribución de Probabilidades:** Visualizar cómo el modelo asigna probabilidades a la clase positiva.

**¿Por qué este paso es importante para un empresario?**
*   **Importancia de Características:** Identificar los factores más influyentes puede guiar estrategias de negocio o intervención. Por ejemplo, si la glucosa es la más importante, se pueden enfocar esfuerzos en el monitoreo de este indicador.
*   **Matriz de Confusión:** Ayuda a entender los tipos de errores que comete el modelo. ¿Está fallando más en predecir diabetes cuando sí la hay (FN), o prediciendo diabetes cuando no la hay (FP)? Esto tiene implicaciones de coste y riesgo.
*   **Curva ROC y AUC:** Ofrecen una visión más matizada del rendimiento que la simple exactitud, especialmente útil para comparar diferentes modelos o configuraciones.
*   **Distribución de Probabilidades:** Entender la confianza del modelo en sus predicciones puede ser útil para establecer umbrales de decisión (e.g., solo actuar sobre predicciones con alta probabilidad).

In [ ]:
# Asegurarnos que 'modelo_rf_base' está entrenado en X_train, y_train
# Si no se ha ejecutado la celda 20 de nuevo después de la validación cruzada, lo re-entrenamos:
# (La celda 20 ya entrena modelo_rf_base en X_train, y_train)

importancias = modelo_rf_base.feature_importances_
nombres_caracteristicas = X_train.columns # Usar X_train.columns para asegurar el orden correcto

# Crear un DataFrame para visualizar mejor las importancias
df_importancias = pd.DataFrame({'Caracteristica': nombres_caracteristicas, 'Importancia': importancias})
df_importancias = df_importancias.sort_values(by='Importancia', ascending=False)

# Visualizar las importancias
plt.figure(figsize=(10,6))
sns.barplot(x='Importancia', y='Caracteristica', data=df_importancias, palette='viridis')
plt.title("Importancia de Variables (Random Forest Base)")
plt.xlabel("Importancia (calculada por Gini impurity)")
plt.ylabel("Característica")
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

print(df_importancias)

**Interpretación de la Importancia de Características:**
`Glucose` es consistentemente la característica más importante, seguida por `BMI`, `Age` y `DiabetesPedigreeFunction`. Esto se alinea con el conocimiento médico y las correlaciones que vimos anteriormente. Saber esto podría, por ejemplo, enfocar campañas de prevención o chequeos médicos en estos factores.

#### 7.2. Matriz de Confusión

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# y_pred_base ya fue calculado en la celda 20 usando modelo_rf_base en X_test
# y_pred_eval = modelo_rf_base.predict(X_test) # Esta línea sería redundante si la celda 20 se ejecutó

cm = confusion_matrix(y_test, y_pred_base, labels=modelo_rf_base.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Diabetes (0)", "Diabetes (1)"])

disp.plot(cmap='Blues')
plt.title("Matriz de Confusión (Modelo Base en Test Set)")
plt.show()

print("Detalle de la Matriz de Confusión:")
print(f"Verdaderos Negativos (TN - No Diabetes, predicho No Diabetes): {cm[0,0]}")
print(f"Falsos Positivos (FP - No Diabetes, predicho Diabetes):      {cm[0,1]}")
print(f"Falsos Negativos (FN - Diabetes, predicho No Diabetes):     {cm[1,0]}")
print(f"Verdaderos Positivos (TP - Diabetes, predicho Diabetes):     {cm[1,1]}")

**Interpretación de la Matriz de Confusión:**
*   **Verdaderos Negativos (TN):** El modelo predijo correctamente "No Diabetes" para [valor de TN] pacientes que realmente no la tienen.
*   **Falsos Positivos (FP):** El modelo predijo incorrectamente "Diabetes" para [valor de FP] pacientes que no la tienen (Error Tipo I).
*   **Falsos Negativos (FN):** El modelo predijo incorrectamente "No Diabetes" para [valor de FN] pacientes que sí la tienen (Error Tipo II). ¡Este es a menudo el error más costoso en contextos médicos!
*   **Verdaderos Positivos (TP):** El modelo predijo correctamente "Diabetes" para [valor de TP] pacientes que sí la tienen.

El objetivo es maximizar TP y TN, y minimizar FP y FN. La importancia relativa de minimizar FP vs. FN depende del problema de negocio.
(Nota: Reemplazar `[valor de TN]`, `[valor de FP]`, `[valor de FN]`, `[valor de TP]` con los números reales de la salida de la celda anterior después de ejecutar el notebook).

#### 7.3. Curva ROC y Puntuación AUC

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_auc_score

# Obtener las probabilidades de predicción para la clase positiva (Diabetes) usando modelo_rf_base
y_proba_base = modelo_rf_base.predict_proba(X_test)[:,1]

RocCurveDisplay.from_estimator(modelo_rf_base, X_test, y_test)
plt.title("Curva ROC (Modelo Base en Test Set)")
plt.plot([0, 1], [0, 1], 'k--', label='Clasificador Aleatorio') # Línea de referencia
plt.legend()
plt.show()

auc_score_base = roc_auc_score(y_test, y_proba_base)
print(f"Puntuación AUC (Area Under Curve) del Modelo Base: {auc_score_base:.4f}")

**Interpretación de la Curva ROC y AUC:**
La curva ROC muestra qué tan bien el modelo distingue entre las dos clases. Una curva que se acerca más a la esquina superior izquierda indica un mejor rendimiento. El AUC es el área bajo esta curva. Un AUC de 0.5 representa un clasificador aleatorio (sin capacidad de discriminación), mientras que un AUC de 1.0 representa un clasificador perfecto.
Nuestro modelo base obtiene un AUC de [valor de AUC], lo que indica una capacidad de discriminación [buena/moderada/aceptable]. Hay espacio para mejorar, pero es significativamente mejor que el azar.
(Nota: Reemplazar `[valor de AUC]` y la descripción con el valor real de la salida de la celda anterior después de ejecutar).

#### 7.4. Distribución de Probabilidades Predichas

In [ ]:
# y_proba_base ya fue calculado en la celda anterior

plt.figure(figsize=(10, 6))
sns.histplot(y_proba_base, bins=30, kde=True)
plt.title("Distribución de Probabilidades Predichas de Tener Diabetes (Clase 1) - Modelo Base")
plt.xlabel("Probabilidad Predicha de Diabetes")
plt.ylabel("Frecuencia")
plt.axvline(0.5, color='red', linestyle='--', label='Umbral 0.5') # Umbral de decisión por defecto
plt.legend()
plt.show()

**Interpretación de la Distribución de Probabilidades:**
Este histograma muestra las probabilidades que el modelo asignó a cada paciente del conjunto de prueba para la clase "Diabetes". Idealmente, para pacientes que realmente no tienen diabetes, las probabilidades deberían estar cerca de 0, y para los que sí tienen, cerca de 1.
Vemos que hay una superposición de distribuciones alrededor del umbral de 0.5, lo que explica por qué el modelo comete errores. Ajustar este umbral podría ser una estrategia para optimizar según si preferimos minimizar Falsos Positivos o Falsos Negativos.

### 8. Optimización de Hiperparámetros

Los modelos de Machine Learning tienen "hiperparámetros" que no se aprenden de los datos, sino que se configuran antes del entrenamiento (e.g., el número de árboles en un Random Forest). Encontrar la combinación óptima de hiperparámetros puede mejorar significativamente el rendimiento del modelo.

Utilizaremos dos técnicas comunes:
1.  **GridSearchCV:** Prueba todas las combinaciones posibles de un conjunto predefinido de hiperparámetros y selecciona la mejor según una métrica de evaluación (e.g., F1-score) usando validación cruzada.
2.  **RandomizedSearchCV:** En lugar de probar todas las combinaciones, muestrea aleatoriamente un número fijo de combinaciones de un espacio de búsqueda de hiperparámetros. Puede ser más eficiente que GridSearchCV cuando el espacio de búsqueda es grande.

**Importante:** La optimización de hiperparámetros se realiza sobre el **conjunto de entrenamiento** (`X_train`, `y_train`). El conjunto de prueba (`X_test`) se mantiene reservado para la evaluación final del modelo optimizado.

**¿Por qué este paso es importante para un empresario?**
Esto es como ajustar finamente una máquina para obtener el mejor rendimiento. Un modelo bien afinado puede ofrecer predicciones más precisas y fiables, lo que se traduce en mejores decisiones de negocio y mayor retorno de la inversión en la solución de IA.

In [ ]:
# Importar GridSearchCV
from sklearn.model_selection import GridSearchCV

# Definir el espacio de búsqueda de hiperparámetros para RandomForestClassifier
parametros_grid = {
    'n_estimators': [50, 100, 150],       # Número de árboles en el bosque
    'max_depth': [None, 10, 20],        # Profundidad máxima de cada árbol
    'min_samples_split': [2, 5, 10],   # Número mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4]      # Número mínimo de muestras en un nodo hoja
}

# Inicializar GridSearchCV
# estimator: el modelo a optimizar (RandomForestClassifier)
# param_grid: el diccionario de hiperparámetros a probar
# cv: la estrategia de validación cruzada (usaremos StratifiedKFold con 5 splits)
# scoring: la métrica para evaluar las combinaciones (usaremos 'f1' por el desbalanceo)
# n_jobs=-1: usar todos los procesadores disponibles para acelerar la búsqueda
grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                           param_grid=parametros_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), # Usar StratifiedKFold explícitamente
                           scoring="f1",
                           n_jobs=-1,
                           verbose=1) # verbose para ver el progreso

# Ejecutar la búsqueda de hiperparámetros sobre los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados y el mejor score F1
print("\nMejores parámetros encontrados (GridSearchCV):", grid_search.best_params_)
print("Mejor F1-score en validación cruzada (GridSearchCV):", grid_search.best_score_)

# El mejor modelo ya está entrenado y se puede acceder con grid_search.best_estimator_
modelo_optimizado_grid = grid_search.best_estimator_

#### 8.2. RandomizedSearchCV (Búsqueda Aleatoria)

In [ ]:
# Importar RandomizedSearchCV y distribuciones para parámetros continuos o discretos grandes
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definir el espacio de búsqueda de hiperparámetros usando distribuciones
parametros_random = {
    'n_estimators': randint(50, 250), # Entero aleatorio entre 50 y 249
    'max_depth': [None] + list(range(5, 25, 5)), # None o valores discretos 5, 10, 15, 20
    'min_samples_split': randint(2, 11), # Entero aleatorio entre 2 y 10
    'min_samples_leaf': randint(1, 5)    # Entero aleatorio entre 1 y 4
}

# Inicializar RandomizedSearchCV
# n_iter: número de combinaciones de parámetros a probar (e.g., 50 o 100)
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42),
                                   param_distributions=parametros_random,
                                   n_iter=50, # Probar 50 combinaciones aleatorias
                                   cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), # Usar StratifiedKFold
                                   scoring="f1",
                                   n_jobs=-1,
                                   random_state=42, # Para reproducibilidad de la búsqueda aleatoria
                                   verbose=1)

# Ejecutar la búsqueda aleatoria de hiperparámetros
random_search.fit(X_train, y_train)

# Mostrar los mejores parámetros y el mejor score F1
print("\nMejores parámetros encontrados (RandomizedSearchCV):", random_search.best_params_)
print("Mejor F1-score en validación cruzada (RandomizedSearchCV):", random_search.best_score_)

# El mejor modelo ya está entrenado y se puede acceder con random_search.best_estimator_
modelo_optimizado_random = random_search.best_estimator_

**Comentario sobre Optimización:**
Ambas técnicas nos proporcionan un conjunto de hiperparámetros que, según la validación cruzada en los datos de entrenamiento, ofrecen el mejor F1-score. Generalmente, se elige el `best_estimator_` de la búsqueda que haya dado un mejor `best_score_` para la evaluación final en el `X_test`.

### 9. Pipeline de Machine Learning Completo con Preprocesamiento y Optimización

Una práctica recomendada es integrar los pasos de preprocesamiento (como el escalado de características) dentro del pipeline de optimización. Esto asegura que el preprocesamiento se aplique correctamente dentro de cada fold de la validación cruzada, evitando la fuga de datos.

1.  **StandardScaler:** Estandariza las características eliminando la media y escalando a la varianza unitaria. Aunque los árboles de decisión (como Random Forest) no son sensibles a la escala de las características, es una buena práctica incluirlo, especialmente si se planea probar otros modelos que sí lo son (e.g., SVM, Regresión Logística).
2.  **Pipeline:** Combinaremos `StandardScaler` y `RandomForestClassifier`.
3.  **GridSearchCV (o RandomizedSearchCV):** Se aplicará al pipeline completo, optimizando los hiperparámetros del modelo *dentro* del pipeline.

**¿Por qué este paso es importante para un empresario?**
Esto demuestra un enfoque riguroso y robusto para construir modelos. Un pipeline bien estructurado no solo mejora la fiabilidad del modelo, sino que también facilita su despliegue y mantenimiento en entornos de producción, ya que todos los pasos de transformación de datos están encapsulados.

In [ ]:
# Importar Pipeline de sklearn y StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Crear el pipeline: primero escalar, luego aplicar el clasificador
pipeline_completo = Pipeline([
    ('scaler', StandardScaler()), # Paso 1: Estandarizar características
    ('modelo_rf', RandomForestClassifier(random_state=42)) # Paso 2: Modelo Random Forest
])

# Definir el espacio de búsqueda de hiperparámetros para el pipeline
# Notar el prefijo 'modelo_rf__' para indicar que estos parámetros pertenecen al paso 'modelo_rf' del pipeline
parametros_pipeline = {
    'modelo_rf__n_estimators': [100, 150], # Menos opciones para una ejecución más rápida de ejemplo
    'modelo_rf__max_depth': [None, 10, 20],
    'modelo_rf__min_samples_split': [2, 5],
    'modelo_rf__min_samples_leaf': [1, 2]
}

# Inicializar GridSearchCV para el pipeline completo
grid_search_pipeline = GridSearchCV(pipeline_completo,
                                  param_grid=parametros_pipeline,
                                  cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), # Usar StratifiedKFold
                                  scoring='f1',
                                  n_jobs=-1,
                                  verbose=1)

# Ejecutar la búsqueda sobre los datos de entrenamiento
grid_search_pipeline.fit(X_train, y_train)

print("\nMejor F1-score con pipeline y GridSearchCV:", grid_search_pipeline.best_score_)
print("Mejores parámetros para el pipeline:", grid_search_pipeline.best_params_)

# El mejor pipeline (con escalador ajustado y modelo optimizado) está en:
mejor_pipeline_final = grid_search_pipeline.best_estimator_

# Evaluar el mejor pipeline en el conjunto de prueba
y_pred_pipeline = mejor_pipeline_final.predict(X_test)
print("\nReporte de Clasificación del Mejor Pipeline en el Conjunto de Prueba:")
print(classification_report(y_test, y_pred_pipeline))

# También podemos ver el AUC del pipeline final
y_proba_pipeline = mejor_pipeline_final.predict_proba(X_test)[:,1]
auc_pipeline_final = roc_auc_score(y_test, y_proba_pipeline)
print(f"\nAUC del Mejor Pipeline en el Conjunto de Prueba: {auc_pipeline_final:.4f}")

**Interpretación del Pipeline Optimizado:**
El `grid_search_pipeline.best_score_` nos da el F1-score promedio en validación cruzada sobre los datos de entrenamiento. El reporte de clasificación final sobre `X_test` nos indica cómo se espera que este pipeline optimizado funcione en datos completamente nuevos.
Compare este reporte con el del modelo base. ¿Ha mejorado el F1-score para la clase 1 (Diabetes)? ¿Y la precisión o el recall? ¿Y el AUC?
Este `mejor_pipeline_final` es el candidato a ser el modelo productivo.
(Nota: Tendrás que ejecutar el notebook para ver los valores específicos y completar la comparación).

### 10. Guardado y Carga del Modelo Final

Una vez que hemos encontrado nuestro mejor modelo (en este caso, `mejor_pipeline_final`), es importante guardarlo para poder usarlo en el futuro sin necesidad de reentrenarlo cada vez. La librería `joblib` es eficiente para guardar objetos de Python, incluyendo modelos de `scikit-learn`.

**¿Por qué este paso es importante para un empresario?**
Guardar el modelo permite su despliegue en sistemas de producción, donde puede hacer predicciones sobre nuevos datos en tiempo real o por lotes. Es el paso que convierte el trabajo de desarrollo en una herramienta operativa y de valor continuo para la organización.

In [ ]:
import joblib

# Nombre del archivo para guardar el modelo
nombre_archivo_modelo = "pipeline_diabetes_optimizado.pkl"

# Guardar el mejor pipeline (que incluye escalador y modelo RandomForest optimizado)
joblib.dump(mejor_pipeline_final, nombre_archivo_modelo)
print(f"Modelo guardado como {nombre_archivo_modelo}")

# Ejemplo de cómo cargar el modelo posteriormente
modelo_cargado = joblib.load(nombre_archivo_modelo)
print("\nModelo cargado exitosamente.")

# Verificar que el modelo cargado funciona (haciendo una predicción de ejemplo)
# Tomamos la primera fila de X_test como ejemplo
ejemplo_prediccion = modelo_cargado.predict(X_test.head(1))
ejemplo_probabilidad = modelo_cargado.predict_proba(X_test.head(1))

print(f"Predicción para la primera muestra de X_test: {ejemplo_prediccion[0]}")
print(f"Probabilidades para la primera muestra de X_test (No Diabetes, Diabetes): {ejemplo_probabilidad[0]}")

### 11. Conclusiones y Próximos Pasos

**Resumen del Trabajo Realizado:**
A lo largo de este notebook, hemos seguido un proceso completo de desarrollo de un modelo de Machine Learning para la predicción de diabetes:
1.  Cargamos y exploramos el conjunto de datos Pima Indians Diabetes.
2.  Realizamos un preprocesamiento exhaustivo, incluyendo el manejo de valores anómalos (ceros implausibles) y la imputación de datos faltantes.
3.  Visualizamos los datos para entender mejor sus distribuciones y relaciones.
4.  Dividimos los datos en conjuntos de entrenamiento y prueba, utilizando estratificación para manejar el desbalanceo de clases.
5.  Entrenamos un modelo base (Random Forest) y lo evaluamos.
6.  Implementamos estrategias avanzadas como la validación cruzada y SMOTE para obtener una evaluación más robusta y mejorar el manejo del desbalanceo.
7.  Interpretamos el modelo mediante la importancia de características, matriz de confusión y curva ROC/AUC.
8.  Optimizamos los hiperparámetros del modelo utilizando GridSearchCV y RandomizedSearchCV, integrando el preprocesamiento (escalado) en un pipeline completo.
9.  Seleccionamos el mejor pipeline y lo evaluamos en el conjunto de prueba reservado.
10. Guardamos el modelo final para su posible despliegue.

**Resultados Clave:**
*   El modelo final (pipeline con StandardScaler y RandomForest optimizado) alcanzó un F1-score de **[insertar F1-score del `mejor_pipeline_final` en `X_test` para la clase 1]** y un AUC de **[insertar AUC del `mejor_pipeline_final` en `X_test`]** en el conjunto de prueba para la predicción de diabetes.
*   Las características más influyentes para la predicción fueron `Glucose`, `BMI`, y `Age`.
*   Se demostró la importancia de un preprocesamiento cuidadoso y la optimización de hiperparámetros para construir un modelo robusto.
(Nota: Deberás ejecutar el notebook completo y rellenar los valores entre corchetes con los resultados obtenidos en la celda 45).

**¿Qué significa esto para el negocio?**
Hemos desarrollado un prototipo de herramienta analítica que puede ayudar a identificar individuos con mayor riesgo de padecer diabetes. Este modelo:
*   Proporciona una base cuantitativa para la toma de decisiones.
*   Puede integrarse en sistemas de salud para apoyar a los profesionales médicos (nunca para reemplazarlos).
*   Permite enfocar recursos de prevención y tratamiento de manera más eficiente.

**Próximos Pasos y Mejoras Potenciales:**
*   **Probar otros algoritmos:** Comparar el rendimiento con otros modelos como Gradient Boosting, SVM, Regresión Logística, o Redes Neuronales.
*   **Ingeniería de Características Avanzada:** Crear nuevas características a partir de las existentes que puedan capturar relaciones más complejas.
*   **Análisis de Errores más Profundo:** Investigar los casos donde el modelo falla consistentemente para entender sus limitaciones.
*   **Considerar el Costo de los Errores:** Ajustar el umbral de decisión del modelo o usar métricas de evaluación ponderadas por costo si los falsos negativos tienen un impacto mucho mayor que los falsos positivos (o viceversa).
*   **Monitoreo del Modelo (MLOps):** Si se despliega, implementar un sistema para monitorear su rendimiento a lo largo del tiempo y reentrenarlo cuando sea necesario.
*   **Validación Externa:** Probar el modelo en un conjunto de datos completamente diferente (de otra población u hospital) para evaluar su generalizabilidad.

Este proyecto demuestra la capacidad de aplicar técnicas de ciencia de datos para abordar problemas del mundo real y generar valor. La metodología seguida es estándar en la industria y sienta las bases para desarrollos más avanzados.